# Installing Transformers and importing Dependencies

In [1]:
from bs4 import BeautifulSoup
from transformers import pipeline
import requests
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [93]:
summarizer=pipeline("summarization")

No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [5]:
# summarizer(article,max_length=130,min_length=30,do_sample=False)

[{'summary_text': 'chatbots can handle a barrage of customer queries and messages without slowdown . they use techniques backed with Artificial Intelligence, Machine Learning, and Data Science . to train the chatbot, you can use Recurrent Neural Networks .'}]

# Getting Blogpost from Medium

In [133]:
URL = "https://www.therandomblogs.in/2021/11/what-is-metaverse-.html"

In [134]:
r = requests.get(URL)

In [135]:
r

<Response [200]>

In [136]:
r.text

'<!DOCTYPE html>\n<html class=\'ltr\' dir=\'ltr\' xmlns=\'http://www.w3.org/1999/xhtml\' xmlns:b=\'http://www.google.com/2005/gml/b\' xmlns:data=\'http://www.google.com/2005/gml/data\' xmlns:expr=\'http://www.google.com/2005/gml/expr\'>\n<head>\n<script async=\'async\' data-ad-client=\'ca-pub-5067184165445260\' src=\'https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js\'></script>\n<meta content=\'width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1\' name=\'viewport\'/>\n<title>What is metaverse? Why Facebook changed its name to Meta?</title>\n<meta content=\'text/html; charset=UTF-8\' http-equiv=\'Content-Type\'/>\n<!-- Chrome, Firefox OS and Opera -->\n<meta content=\'#f8f8f8\' name=\'theme-color\'/>\n<!-- Windows Phone -->\n<meta content=\'#f8f8f8\' name=\'msapplication-navbutton-color\'/>\n<meta content=\'blogger\' name=\'generator\'/>\n<link href=\'https://www.therandomblogs.in/favicon.ico\' rel=\'icon\' type=\'image/x-icon\'/>\n<link href=\'https://www

In [137]:
soup = BeautifulSoup(r.text,"html.parser")
results = soup.find_all(["h1","p"])

# Getting all Text from the Results

In [138]:
text = [result.text for result in results]
article = " ".join(text)

In [139]:
article

"\nWhat is metaverse? Why Facebook changed its name to Meta?\n Metaverse and Facebook's transition to Meta is one of the hottest topics in the tech industry right now. Meta and Metaverse are two terms that every techie has heard of. So, what exactly is a metaverse, how does it work, and how will it affect the world? Here's everything you need to know about the metaverse, including its benefits and drawbacks, as well as why Facebook changed its name to Meta.    What is Metaverse? This metaverse concept is being developed by major tech companies such as Tesla, NVIDIA, Epic games, Microsoft, as well as Meta (Facebook). Let’s understand in simple terms first, metaverse is nothing but a half real, half fictitious world where you can live in a virtual world by just wearing a VR headset.\xa0 You may have heard of or played open world video games such as GTA V, Watchdog, Fortnite, and many others. These games are played on a television, computer, or mobile device. You control the game via cons

# Creating a chunk of text as there is a limit to how much we can pass to a summarizer
## As the maximum length of the input sentence can be 512 words so we need to create chunks of text.

In [140]:
# Now splitting the articles in the for of sentences.
## now changing the full stops and puncuations into <eos>

In [141]:
article = article.replace(".",".<eos>")
article = article.replace("!","!<eos>")
article = article.replace("?","?<eos>")
#article= article.replace("\n","<eos>")
sentences = article.split("<eos>")

In [142]:
sentences

['\nWhat is metaverse?',
 ' Why Facebook changed its name to Meta?',
 "\n Metaverse and Facebook's transition to Meta is one of the hottest topics in the tech industry right now.",
 ' Meta and Metaverse are two terms that every techie has heard of.',
 ' So, what exactly is a metaverse, how does it work, and how will it affect the world?',
 " Here's everything you need to know about the metaverse, including its benefits and drawbacks, as well as why Facebook changed its name to Meta.",
 '    What is Metaverse?',
 ' This metaverse concept is being developed by major tech companies such as Tesla, NVIDIA, Epic games, Microsoft, as well as Meta (Facebook).',
 ' Let’s understand in simple terms first, metaverse is nothing but a half real, half fictitious world where you can live in a virtual world by just wearing a VR headset.',
 '\xa0 You may have heard of or played open world video games such as GTA V, Watchdog, Fortnite, and many others.',
 ' These games are played on a television, comput

In [143]:
# no more then 500 words in one go
# using each word as it makes it easier to count the number of words in the list

max_chunk=450
current_chunk=0
chunks=[]

for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(" ")) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(" "))
        else:
            current_chunk +=1
            chunks.append(sentence.split(" "))
    else:
        print(current_chunk)
        chunks.append(sentence.split(" "))
        pass


0


In [144]:
# Now joining all the splitted words together
for i in range(len(chunks)):
    chunks[i] = " ".join(chunks[i])

In [145]:
chunks[0]

"\nWhat is metaverse?  Why Facebook changed its name to Meta? \n Metaverse and Facebook's transition to Meta is one of the hottest topics in the tech industry right now.  Meta and Metaverse are two terms that every techie has heard of.  So, what exactly is a metaverse, how does it work, and how will it affect the world?  Here's everything you need to know about the metaverse, including its benefits and drawbacks, as well as why Facebook changed its name to Meta.     What is Metaverse?  This metaverse concept is being developed by major tech companies such as Tesla, NVIDIA, Epic games, Microsoft, as well as Meta (Facebook).  Let’s understand in simple terms first, metaverse is nothing but a half real, half fictitious world where you can live in a virtual world by just wearing a VR headset. \xa0 You may have heard of or played open world video games such as GTA V, Watchdog, Fortnite, and many others.  These games are played on a television, computer, or mobile device.  You control the ga

In [146]:

res = summarizer(chunks,max_length=150,min_lenght=30,do_sample=False)

In [147]:
for i in res:
    print(i["summary_text"])
    print("\n")

metaverse is a half real, half fictitious world where you can live in a virtual world by just wearing a VR headset . you can meet up with friends, play outdoor games, or attend parties from the comfort of our own room . in the metaverse, you can dress up like you and have facial expressions .


augmented reality (AR) and virtual reality (VR) can help you create a virtual world around you . it can be accessed via smartphone cameras or other devices such as smart glasses and smart cameras to play games and for presentations . the metaverse is a little different and more advanced than AR . people can become disconnected from reality in the same way as video games .


facebook/Meta has been chastised for how unsafe users' data is on Facebook, as well as how it handles user data . the move may help it in overcoming this bad image and establish a fresh and positive reputation .


